In [1]:
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import word_tokenize as WordTokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
import pickle 

In [2]:
# set the path of all files
spam_train_files = r'data\spam-train\*.txt'
not_spam_train_files=r'data\nonspam-train\*.txt'

# method to get files data from disk
def get_DatafromFiles(file_path):
    filesList = []
    file_data_list=[]
    for files in glob.glob(file_path):
        filesList.append(files) 
    for file in filesList:
        with open(file,'r') as curfile:
            file_data_list.append(curfile.readline())
    return file_data_list
 
# prepare the traning data
spam_train_df=pd.DataFrame(get_DatafromFiles(spam_train_files),columns=['email'])
spam_train_df['email_class']='spam'

not_spam_train_df=pd.DataFrame(get_DatafromFiles(not_spam_train_files),columns=['email'])
not_spam_train_df['email_class']='not-spam'
train_df=not_spam_train_df.append(spam_train_df)
train_df.reset_index(drop=True,inplace=True)

In [3]:
#genrate X and y
X=pd.DataFrame(train_df['email'])
y=pd.DataFrame(train_df['email_class'])

In [4]:
#split the test and train data for validation
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.30, random_state=42)

In [5]:
#method to genrate tokens
def word_tokenizer(data):
    return str(nltk.word_tokenize(data))

In [6]:
# get the tokens for train and test
X_tokenized =X['email'].apply(lambda x:word_tokenizer(x))
X_train_tokenized =X_train['email'].apply(lambda x:word_tokenizer(x))
X_test_tokenized =X_test['email'].apply(lambda x:word_tokenizer(x))

In [7]:
#genrate vocabulary from corpus andf get vectors for train and test
tfidf_vectorizer =TfidfVectorizer(stop_words='english',lowercase=True,ngram_range=(1, 2),max_features=5000)
X_vec=tfidf_vectorizer.fit(X_tokenized)
X_train_vec=tfidf_vectorizer.transform(X_train_tokenized)
X_test_vec=tfidf_vectorizer.transform(X_test_tokenized)

In [8]:
#build the model for classification LogisticRegression
Classifier=LogisticRegression()
Classifier.fit(X_train_vec,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [12]:
#get the prediction for train and test set and show the accuracy for both
predict_train=pd.DataFrame(Classifier.predict(X_train_vec))
predict_test=pd.DataFrame(Classifier.predict(X_test_vec))
print("accuracy for train set={0}".format(accuracy_score(y_train,predict_train)))
print("accuracy for test set={0}".format(accuracy_score(y_test,predict_test)))

accuracy for train set=0.9959183673469387
accuracy for test set=0.9904761904761905


In [13]:
#perform cross_validate
cv_results = cross_validate(Classifier, X_test_vec, y_test, cv=10)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

In [14]:
#check cross validation result
cv_results['test_score']

array([ 1.        ,  0.95454545,  0.95238095,  0.9047619 ,  1.        ,
        1.        ,  0.95238095,  1.        ,  1.        ,  1.        ])

In [15]:
#save the TfidfVectorizer for prediction
TfidfVectorizer_pickle_file=r'models\TfidfVectorizer_pickle.dat'
TfidfVectorizer_pickle = open(TfidfVectorizer_pickle_file, 'wb')
pickle.dump(tfidf_vectorizer,TfidfVectorizer_pickle)
TfidfVectorizer_pickle.close()

In [16]:
#save the model for prediction
Classifier_pickle_file=r'models\Classifier_pickle.dat'
Classifier_pickle = open(Classifier_pickle_file, 'wb')
pickle.dump(Classifier,Classifier_pickle)
Classifier_pickle.close()